## Sentiment Analysis

### Text

- textblob.sentiments: NaiveBayesAnalyzer, PatternAnalyzer
    - result: polarity [-1,1], subjectivity [-1,1]

- VADER: 

In [4]:
%pip install vaderSentiment
%pip install textblob

  Obtaining dependency information for vaderSentiment from https://files.pythonhosted.org/packages/76/fc/310e16254683c1ed35eeb97386986d6c00bc29df17ce280aed64d55537e9/vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.7 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
from preprocessing.preprocessing import *

In [3]:
# sentiment analysis imports

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [9]:
df = load_dataset(subset_name="climatevisions_2019_popular")
df = preprocess_dataset(df, selected_columns=None)

Items: tweet_data
(5000, 9)


In [6]:
df.head()

,created_at,img_name,language,like_count,quote_count,referenced_tweets,retweet_count,text,tweet_id
0,2019-07-22T12:38:24.000Z,id_1153283149360762880_2019-07-22.jpg\n,en,b'82582',b'3918',<NA>,b'50280',the UN released a 740 page report compiled ove...,1153283149360762880
1,2019-08-20T09:28:39.000Z,id_1163744643600637952_2019-08-20.jpg\n,en,b'69820',b'2456',<NA>,b'51781',"The Amazon Rainforest, one of the wettest plac...",1163744643600637952
2,2019-04-28T18:51:22.000Z,id_1122574040936452097_2019-04-28.jpg\n,en,b'69235',b'87',<NA>,b'11051',just learned about climate change https://t.co...,1122574040936452097
3,2019-10-28T13:10:13.000Z,id_1188805167958974465_2019-10-28.jpg\n,en,b'65465',b'70',<NA>,b'6124',Climate change caused this. https://t.co/JG2Ly...,1188805167958974465
4,2019-03-19T16:30:00.000Z,id_1108042949449969666_2019-03-19.jpg\n,en,b'62852',b'976',<NA>,b'9145',#GreenNewDeal haters\xe2\x80\x99 plan to addre...,1108042949449969666


In [7]:
df.isna().sum()

created_at           0
img_name             0
language             0
like_count           0
quote_count          0
referenced_tweets    0
retweet_count        0
text                 0
tweet_id             0
dtype: int64

In [ ]:
# textblob
# from textblob import TextBlob
# from textblob.sentiments import NaiveBayesAnalyzer, PatternAnalyzer

# text = "That is really dangerous"
# print(TextBlob(text, analyzer=NaiveBayesAnalyzer()).sentiment)
# print(TextBlob(text, analyzer=PatternAnalyzer()).sentiment)

In [17]:
# VADER
df_sentiments = df.copy()

sentiment = SentimentIntensityAnalyzer()

df_sentiments['pos'] = [sentiment.polarity_scores(x)['pos'] for x in df_sentiments['text']]
df_sentiments['neu'] = [sentiment.polarity_scores(x)['neu'] for x in df_sentiments['text']]
df_sentiments['neg'] = [sentiment.polarity_scores(x)['neg'] for x in df_sentiments['text']]




,created_at,img_name,language,like_count,quote_count,referenced_tweets,retweet_count,text,tweet_id
4995,2019-05-06T19:22:03.000Z,id_1125480867403636737_2019-05-06.jpg\n,en,b'148',b'4',<NA>,b'81',Only 3.5% of national television news segments...,1125480867403636737
4996,2019-05-08T15:04:33.000Z,id_1126140840882130946_2019-05-08.jpg\n,de,b'148',b'12',<NA>,b'48',\xf0\x9f\x9a\xa8 Es wird Zeit zu handeln! M\xc...,1126140840882130946
4997,2019-05-15T20:33:28.000Z,id_1128760329830146048_2019-05-15.jpg\n,en,b'148',b'2',<NA>,b'83',Hello everyone. One big reminder before the el...,1128760329830146048
4998,2019-05-21T19:11:58.000Z,id_1130914144922812416_2019-05-21.jpg\n,en,b'148',b'19',<NA>,b'85',I\xe2\x80\x99ve argued for the importance of c...,1130914144922812416
4999,2019-05-24T16:47:21.000Z,id_1131964914136965120_2019-05-24.jpg\n,en,b'148',b'6',<NA>,b'46',Kids can make sceptical parents change their m...,1131964914136965120


In [18]:
for i in range(10):
    text = df['text'][i]
    print(text)
    sentiment = SentimentIntensityAnalyzer()
    result = sentiment.polarity_scores(text)
    for k, v in result.items():
        print(k,v)

the UN released a 740 page report compiled over six years by 250 scientists from 70 nations that basically states that unless drastic measures are taken to combat climate change, we are looking at mass extinction by 2050. At this point I don\xe2\x80\x99t know if having kids is even ethical https://t.co/CsdTSpfRPv
neg 0.044
neu 0.896
pos 0.06
compound 0.2263
The Amazon Rainforest, one of the wettest places on Earth, is on fire.  Siberia, one of the coldest places on Earth, is on fire.That aint normal. Climate change is for real and our planet is burning up. Humans are the cause of this deadly fever.#PrayforAmazonia https://t.co/D6VCTaCNXt
neg 0.05
neu 0.915
pos 0.035
compound -0.1779
just learned about climate change https://t.co/OlmKwz2bOn
neg 0.0
neu 1.0
pos 0.0
compound 0.0
Climate change caused this. https://t.co/JG2LyeX8Tx
neg 0.0
neu 1.0
pos 0.0
compound 0.0
#GreenNewDeal haters\xe2\x80\x99 plan to address Climate Change: https://t.co/ahuDXnxcDG
neg 0.0
neu 1.0
pos 0.0
compound 0.

In [19]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
text = df['text'][0]
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(output)

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.6202,  0.4275, -2.5437]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


### Images